# Import & Setup

In [23]:
import pandas as pd
import numpy as np
import json
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)

In [24]:
df_all = pd.read_csv('LazyVoiceFinder_export.csv', low_memory=False)
df_all = df_all.loc[~(df_all['State'].isin(['Bad File Name, No Dialogue','No Voice File']))]
df_all = df_all.loc[(df_all['Dialogue 1 - English'].notna()) & (df_all['Dialogue 1 - English']!='')]
print(df_all.shape)
df_all.head()

(74642, 31)


,State,Index,Plugin,Last Modifier,FormId,FormId without Load#,Response Num,Edid,Topic FormId,Topic Edid,Topic Text,Quest FormId,Quest Edid,Quest Name,Category/Subtype,Emotion,Conditions,Bsa Name,Full Path,Base Path,Path,File Name,Extensions,Voice Type,Prompt,Dialogue 1 - English,Dialogue 2 - English,Script Notes,Edits,Keyword,StringId
26,NaN,0,skyrim.esm,skyrim.esm,000130DA,000130DA,1.0,[000130DA],00035E10,MG07TolfdirForcegreetBranchTopic,NaN,0001F257,MG07,The Staff of Magnus,Topic/Custom,Neutral,"GetStage: ""The Staff of Magnus"" [QUST:0001F257] >= 50\nGetIsID: ""Tolfdir"" [NPC_:0001C19E] = 1",Skyrim - Voices_en0.bsa,E:\SteamLibrary\steamapps\common\Skyrim Special Edition\Data\Skyrim - Voices_en0.bsa\sound\voice\skyrim.esm\maleoldkindly\mg07_mg07tolfdirforcegreet_000130da_1.fuz,E:\SteamLibrary\steamapps\common\Skyrim Special Edition\Data\Skyrim - Voices_en0.bsa,sound\voice\skyrim.esm\maleoldkindly\mg07_mg07tolfdirforcegreet_000130da_1.fuz,mg07_mg07tolfdirforcegreet_000130da_1.fuz,.fuz,maleoldkindly,NaN,"You survived! You have it, then?","You survived! You have it, then?",NaN,NaN,NaN,25001.0
27,NaN,0,skyrim.esm,skyrim.esm,000130DA,000130DA,2.0,[000130DA],00035E10,MG07TolfdirForcegreetBranchTopic,NaN,0001F257,MG07,The Staff of Magnus,Topic/Custom,Neutral,"GetStage: ""The Staff of Magnus"" [QUST:0001F257] >= 50\nGetIsID: ""Tolfdir"" [NPC_:0001C19E] = 1",Skyrim - Voices_en0.bsa,E:\SteamLibrary\steamapps\common\Skyrim Special Edition\Data\Skyrim - Voices_en0.bsa\sound\voice\skyrim.esm\maleoldkindly\mg07_mg07tolfdirforcegreet_000130da_2.fuz,E:\SteamLibrary\steamapps\common\Skyrim Special Edition\Data\Skyrim - Voices_en0.bsa,sound\voice\skyrim.esm\maleoldkindly\mg07_mg07tolfdirforcegreet_000130da_2.fuz,mg07_mg07tolfdirforcegreet_000130da_2.fuz,.fuz,maleoldkindly,NaN,Let's hope it's as powerful as the Psijics believe it to be.,Let's hope it's as powerful as the Psijics believe it to be.,NaN,NaN,NaN,17544.0
28,NaN,0,skyrim.esm,skyrim.esm,00013113,00013113,1.0,[00013113],0001310D,[0001310D],NaN,00036192,MQ203,Alduin's Wall,Scene/Scene,Surprise,NaN,Skyrim - Voices_en0.bsa,E:\SteamLibrary\steamapps\common\Skyrim Special Edition\Data\Skyrim - Voices_en0.bsa\sound\voice\skyrim.esm\femaleuniquedelphine\mq203__00013113_1.fuz,E:\SteamLibrary\steamapps\common\Skyrim Special Edition\Data\Skyrim - Voices_en0.bsa,sound\voice\skyrim.esm\femaleuniquedelphine\mq203__00013113_1.fuz,mq203__00013113_1.fuz,.fuz,femaleuniquedelphine,NaN,"That's done it! Look, it's coming to life!","That's done it! Look, it's coming to life!",awe breaking through Delphine's usual jaded persona,NaN,NaN,60023.0
29,NaN,0,skyrim.esm,skyrim.esm,00013114,00013114,1.0,[00013114],0001310C,[0001310C],NaN,00036192,MQ203,Alduin's Wall,Scene/Scene,Happy,NaN,Skyrim - Voices_en0.bsa,E:\SteamLibrary\steamapps\common\Skyrim Special Edition\Data\Skyrim - Voices_en0.bsa\sound\voice\skyrim.esm\femaleuniquedelphine\mq203__00013114_1.fuz,E:\SteamLibrary\steamapps\common\Skyrim Special Edition\Data\Skyrim - Voices_en0.bsa,sound\voice\skyrim.esm\femaleuniquedelphine\mq203__00013114_1.fuz,mq203__00013114_1.fuz,.fuz,femaleuniquedelphine,NaN,You did it. There's the entrance!,You did it. There's the entrance!,NaN,NaN,NaN,49515.0
30,NaN,0,skyrim.esm,skyrim.esm,00013115,00013115,1.0,[00013115],0001310A,[0001310A],NaN,00036192,MQ203,Alduin's Wall,Scene/Scene,Surprise,NaN,Skyrim - Voices_en0.bsa,E:\SteamLibrary\steamapps\common\Skyrim Special Edition\Data\Skyrim - Voices_en0.bsa\sound\voice\skyrim.esm\maleuniqueesbern\mq203__00013115_1.fuz,E:\SteamLibrary\steamapps\common\Skyrim Special Edition\Data\Skyrim - Voices_en0.bsa,sound\voice\skyrim.esm\maleuniqueesbern\mq203__00013115_1.fuz,mq203__00013115_1.fuz,.fuz,maleuniqueesbern,NaN,There's no telling what we might find inside!,There's no telling what we might find inside!,excitedly,NaN,NaN,30760.0


In [25]:
df_all['Topic Text'].value_counts().head(5)

Topic Text
DialogueGenericSharedInfo      2010
CWSharedInfoStack               978
Hello.                          376
ShoutStartShort                 365
What have you got for sale?     354
Name: count, dtype: int64

In [26]:
df_all['Prompt'].value_counts().head(5)

Prompt
(Remain silent)                                            77
I'd like to decorate my home.                              48
Consider it done.                                          41
What'll you give me for these?                             28
I'll take you to the monster you're hunting. (Persuade)    24
Name: count, dtype: int64

In [27]:
filtered_cols = [
    'FormId', 'Response Num', 'Topic Text', 'Quest FormId', 'Quest Name',
    'Category/Subtype', 'Emotion', 'Voice Type', 'Prompt', 'Dialogue 1 - English'
]

# Filter relevant columns from the DataFrame
df = df_all[filtered_cols]

# Filter rows where either 'Topic Text' or 'Prompt' are not NaN
df = df[df['Topic Text'].notna() | df['Prompt'].notna()]

# Further filter rows where 'Quest Name' is not 'Generic dialogue'
df = df[df['Quest Name'] != 'Generic dialogue']

# Remove rows with excluded topics, as the responses from these vary too wildly
excluded_topics = [
    'Hello.', 'ShoutStartShort', 'ShoutStartLong', 'PushRoDa', 'ShoutEndShort', 'DCETAttack',
    'RelationshipAdoption_SharedInfos', '(Invisible continue)', '(Invisible Continue)', '...', 'Shout2', 'Shout1b',
    'CommandedHello', 'MS11', 'DLC1ThrallHellos', 'DialogueCarriageSystemIdle', 'DCETNoticeCorpse', '(Sharedinfo)',
    'TG01Hellos', 'TG09NDAttacks', 'RelationshipMarriagePostWeddingLoveInterestBlockingTopic', 'DA03Hello',
    '(Invisible Continues Linked to Self)', 'TG08BHellos', 'BYOHRelationshipAdoptableOrphanable_SharedInfos',
    'DLC1VQ07Attack', '(Invisible Continue) Location of Attack', 'RelationshipAdoption_FGReceiveGiftTopic',
    'DBNazirEvictionBranchTopic', 'DLC1VQ04Hellos', 'dunRannveigQSTSHARED01', 'T01Hellos', 'TG09NDDeaths',
    '(Invisible Continue)(Forced Good bye) Time Limit to Attack', '(Invisible Continue)(Forced Goodbye) Time Limit to Attack',
    '(forcegreet)', '(Invisible Continue) Difficulty based on Attack Dela', '(Invisible Continue) Difficulty based on Attack Delta',
    'TG09NDCombatToNormal', 'DB11AmaundJobDoneTopic', 'DB11EmperorPlayerResponse13'
]

df = df[~df['Topic Text'].isin(excluded_topics)]

# Replace 'DialogueRiftenGoodbyes' with 'Goodbye.' in 'Topic Text'
df['Topic Text'] = df['Topic Text'].replace('DialogueRiftenGoodbyes', 'Goodbye.')

# Create a new 'Input' column by prioritizing 'Prompt' over 'Topic Text'
df['Input'] = df.apply(lambda row: row['Prompt'] if pd.notna(row['Prompt']) else row['Topic Text'], axis=1)

df_all.head(5)


,State,Index,Plugin,Last Modifier,FormId,FormId without Load#,Response Num,Edid,Topic FormId,Topic Edid,Topic Text,Quest FormId,Quest Edid,Quest Name,Category/Subtype,Emotion,Conditions,Bsa Name,Full Path,Base Path,Path,File Name,Extensions,Voice Type,Prompt,Dialogue 1 - English,Dialogue 2 - English,Script Notes,Edits,Keyword,StringId
26,NaN,0,skyrim.esm,skyrim.esm,000130DA,000130DA,1.0,[000130DA],00035E10,MG07TolfdirForcegreetBranchTopic,NaN,0001F257,MG07,The Staff of Magnus,Topic/Custom,Neutral,"GetStage: ""The Staff of Magnus"" [QUST:0001F257] >= 50\nGetIsID: ""Tolfdir"" [NPC_:0001C19E] = 1",Skyrim - Voices_en0.bsa,E:\SteamLibrary\steamapps\common\Skyrim Special Edition\Data\Skyrim - Voices_en0.bsa\sound\voice\skyrim.esm\maleoldkindly\mg07_mg07tolfdirforcegreet_000130da_1.fuz,E:\SteamLibrary\steamapps\common\Skyrim Special Edition\Data\Skyrim - Voices_en0.bsa,sound\voice\skyrim.esm\maleoldkindly\mg07_mg07tolfdirforcegreet_000130da_1.fuz,mg07_mg07tolfdirforcegreet_000130da_1.fuz,.fuz,maleoldkindly,NaN,"You survived! You have it, then?","You survived! You have it, then?",NaN,NaN,NaN,25001.0
27,NaN,0,skyrim.esm,skyrim.esm,000130DA,000130DA,2.0,[000130DA],00035E10,MG07TolfdirForcegreetBranchTopic,NaN,0001F257,MG07,The Staff of Magnus,Topic/Custom,Neutral,"GetStage: ""The Staff of Magnus"" [QUST:0001F257] >= 50\nGetIsID: ""Tolfdir"" [NPC_:0001C19E] = 1",Skyrim - Voices_en0.bsa,E:\SteamLibrary\steamapps\common\Skyrim Special Edition\Data\Skyrim - Voices_en0.bsa\sound\voice\skyrim.esm\maleoldkindly\mg07_mg07tolfdirforcegreet_000130da_2.fuz,E:\SteamLibrary\steamapps\common\Skyrim Special Edition\Data\Skyrim - Voices_en0.bsa,sound\voice\skyrim.esm\maleoldkindly\mg07_mg07tolfdirforcegreet_000130da_2.fuz,mg07_mg07tolfdirforcegreet_000130da_2.fuz,.fuz,maleoldkindly,NaN,Let's hope it's as powerful as the Psijics believe it to be.,Let's hope it's as powerful as the Psijics believe it to be.,NaN,NaN,NaN,17544.0
28,NaN,0,skyrim.esm,skyrim.esm,00013113,00013113,1.0,[00013113],0001310D,[0001310D],NaN,00036192,MQ203,Alduin's Wall,Scene/Scene,Surprise,NaN,Skyrim - Voices_en0.bsa,E:\SteamLibrary\steamapps\common\Skyrim Special Edition\Data\Skyrim - Voices_en0.bsa\sound\voice\skyrim.esm\femaleuniquedelphine\mq203__00013113_1.fuz,E:\SteamLibrary\steamapps\common\Skyrim Special Edition\Data\Skyrim - Voices_en0.bsa,sound\voice\skyrim.esm\femaleuniquedelphine\mq203__00013113_1.fuz,mq203__00013113_1.fuz,.fuz,femaleuniquedelphine,NaN,"That's done it! Look, it's coming to life!","That's done it! Look, it's coming to life!",awe breaking through Delphine's usual jaded persona,NaN,NaN,60023.0
29,NaN,0,skyrim.esm,skyrim.esm,00013114,00013114,1.0,[00013114],0001310C,[0001310C],NaN,00036192,MQ203,Alduin's Wall,Scene/Scene,Happy,NaN,Skyrim - Voices_en0.bsa,E:\SteamLibrary\steamapps\common\Skyrim Special Edition\Data\Skyrim - Voices_en0.bsa\sound\voice\skyrim.esm\femaleuniquedelphine\mq203__00013114_1.fuz,E:\SteamLibrary\steamapps\common\Skyrim Special Edition\Data\Skyrim - Voices_en0.bsa,sound\voice\skyrim.esm\femaleuniquedelphine\mq203__00013114_1.fuz,mq203__00013114_1.fuz,.fuz,femaleuniquedelphine,NaN,You did it. There's the entrance!,You did it. There's the entrance!,NaN,NaN,NaN,49515.0
30,NaN,0,skyrim.esm,skyrim.esm,00013115,00013115,1.0,[00013115],0001310A,[0001310A],NaN,00036192,MQ203,Alduin's Wall,Scene/Scene,Surprise,NaN,Skyrim - Voices_en0.bsa,E:\SteamLibrary\steamapps\common\Skyrim Special Edition\Data\Skyrim - Voices_en0.bsa\sound\voice\skyrim.esm\maleuniqueesbern\mq203__00013115_1.fuz,E:\SteamLibrary\steamapps\common\Skyrim Special Edition\Data\Skyrim - Voices_en0.bsa,sound\voice\skyrim.esm\maleuniqueesbern\mq203__00013115_1.fuz,mq203__00013115_1.fuz,.fuz,maleuniqueesbern,NaN,There's no telling what we might find inside!,There's no telling what we might find inside!,excitedly,NaN,NaN,30760.0


In [28]:
def is_unwanted_text(text):
    """Detects if a given text is unwanted based on specific criteria.

    Args:
        text: The input text to check.

    Returns:
        True if the text is unwanted, False otherwise.
    """
    # Basic input sanity checks at the top
    if not isinstance(text, str) or not text:
        return True  # Unwanted because it's not text.

    # Define valid ending punctuation marks
    valid_punctuation = {'.', '!', '?'}

    # Check if text ends with valid punctuation
    ends_with_punctuation = text.endswith(tuple(valid_punctuation))

    # Initialize flags
    all_uppercase = text.isupper()
    all_lowercase = text.islower()
    all_digits = text.isdigit()
    letter_number_pattern = False
    
    # Check for letter followed by number
    for i in range(len(text) - 1):
        if text[i].isalpha() and text[i + 1].isdigit():
            letter_number_pattern = True
            break

    if not ends_with_punctuation:
        if (all_uppercase or all_lowercase or all_digits or letter_number_pattern):
            return True

    # Loop to prune out the 'edger cases'.
    just_saw_upper = False
    disqualified = False
    for char in text:
        # Disqualifying conditions: underscores and consecutive majuscules.
        if char == '_':
            return True
        if just_saw_upper and char.isupper():
            disqualified = True

        # Setting conditions
        if char.isupper():
            just_saw_upper = True
        if char in valid_punctuation:
            return False

    # Return False if text passes all checks
    return disqualified  # The text is acceptable

In [29]:
df = df[~df['Topic Text'].apply(is_unwanted_text)]
df_all['Topic Text']
df_all['Topic Text'].value_counts().head(15)

Topic Text
DialogueGenericSharedInfo           2010
CWSharedInfoStack                    978
Hello.                               376
ShoutStartShort                      365
What have you got for sale?          354
ShoutStartLong                       348
PushRoDa                             346
ShoutEndShort                        339
DialogueGenericHello                 244
DCETAttack                           240
What can I do to help?               189
DialogueGenericGoodbye               166
RelationshipAdoption_SharedInfos     164
(Invisible continue)                 140
I'm looking for work.                138
Name: count, dtype: int64

In [30]:
# Save the player / NPC interaction in an Alpaca style data format
data = []
for _, row in df.iterrows():
    entry = {
        "instruction": "Generate dialogue in the style of Skyrim.",
        "input": row['Input'],
        "output": row['Dialogue 1 - English']
    }
    data.append(entry)
    

In [31]:
df_all['Prompt']
df_all['Prompt'].value_counts().head(15)

Prompt
(Remain silent)                                                                    77
I'd like to decorate my home.                                                      48
Consider it done.                                                                  41
What'll you give me for these?                                                     28
I'll take you to the monster you're hunting. (Persuade)                            24
Floor decorations. (400 gold)                                                      24
What can you tell me about yourself?                                               19
You know what you have to do.                                                      19
I don't have time for this. I can get rough with you if I have to. (Intimidate)    18
Who is Aventus?                                                                    18
Only a beast kills without feeling.                                                18
Reporting for duty.                            

In [32]:
# Write the data to a JSON file
with open('skyrim_alpaca_style_dataset.json', 'w') as json_file:
    json.dump(data, json_file, indent=4)
    

In [130]:
unwanted_test_cases = [
    ('CWSharedInfoStack', True),            # Expected: True (unwanted)
    ('Will you remove the curse from this ring?', False),  # Expected: False (wanted)
    ('HelloWorld', True),                   # Expected: True (unwanted)
    ('HELLO', True),                        # Expected: True (unwanted)
    ('hello', True),
    ('Pelinal12.', False),
    ('Xyphen-9 send their regards.', False),
    ('...', False),
    ('Hello.', False),
    ('hello.', False),
    ('Hello world', False),                 # Expected: False (wanted)
    ('', True),                             # Expected: True (unwanted)
    ('12345', True),                        # Expected: True (unwanted)
    ('Hello123', True),                     # Expected: True (unwanted)
    ('Yes!', False),                        # Expected: False (wanted)
    ('No.', False),                         # Expected: False (wanted)
    ('Maybe', False),                       # Expected: False (wanted)
    (None, True),                           # Expected: True (unwanted)
    (123, True),                            # Expected: True (unwanted)
    ('TestStringWithoutSpace', True),       # Expected: True (unwanted)
    ('This is a test string', False),       # Expected: False (wanted)
    ('Why?', False),                        # Expected: False (wanted)
    ('OK', True),                           # Expected: True (unwanted)
    ('GO', True),                           # Expected: True (unwanted)
    ('DialogueGenericSharedInfo', True),    # Expected: True (unwanted)
    ('Yes, clear my bounty. (<Global=GuildCost> gold)', False),  # Expected: False (wanted)
    ('DialogueGenericZKeyObject', True),    # Expected: True (unwanted)
    ('Shout2', True),                       # Expected: True (unwanted)
    ('OpenDoor', True),                     # Expected: True (unwanted)
    ('SaveGame', True),                     # Expected: True (unwanted)
    ('Enter your name:', False),            # Expected: False (wanted)
    ('Paused', False),                      # Expected: False (wanted)
    ('ResumeGame', True),                   # Expected: True (unwanted)
    ('CWSharedInfoStack', True),
    ('VariableXYZ', True),
    ('InitializeComponents', True),
    ('RandomStringWithNoMeaning', True),
    ('StartGame', True),
    ('Please select an option:', False),
    ('Error: File not found.', False),
    ('UserProfileData', True),
    ('Welcome to the application!', False),
    ('ABCD1234', True),
    ('This is a sample sentence.', False),
    ('ProcessData', True),
    ('Thank you for using our service.', False),
    ('LoadGameLevel', True),
    ('Press ENTER to proceed.', False),
    ('InvalidInputError', True),
    ('Congratulations, you have won!', False),
    ('SystemConfigurationSettings', True),
    ('Goodbye!', False),
    ('SaveChanges', True),
    ('UserInputHandler', True),
    ('Please enter your username and password.', False),
    ('SessionExpired', True),
    ('Click here to continue.', False),
    ('DataEncryptionKey', True),
    ('The operation completed successfully.', False),
    ('MainMenu', True),
    ('LevelCompleted', True),
    ('Would you like to save your progress?', False),
    ('AuthenticationFailed', True),
    ('SettingsMenu', True),
    ('Loading...', False),
    ('FileNotFoundException', True),
    ('Your changes have been saved.', False),
    ('NetworkConfigurationProtocol', True),
    ('RestartApplication', True),
    ('Are you sure you want to delete this file?', False),
    ('DebugModeEnabled', True),
    ('InstallationComplete', True),
    ('Press any key to exit.', False),
    ('UnhandledExceptionError', True),
    ('Thank you for playing!', False),
    ('ApplicationStartupSequence', True),
    ('EnterYourName', True),
    ('GamePaused', True),
    ('Please wait while we process your request.', False),
    ('ResourceAllocationFailed', True),
    ('EndOfFileReached', True),
    ('Welcome back, user!', False),
]


In [131]:
for text, expected in unwanted_test_cases:
    result = is_unwanted_text(text)
    print(f"'{text}': {result} (Expected: {expected})")
    if result != expected:
        print(f"!!!!!!!!!!!!!!!!!!!!!!!!!!!Mismatch on '{text}': Expected {expected}, got {result}!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")

'CWSharedInfoStack': True (Expected: True)
'Will you remove the curse from this ring?': False (Expected: False)
'HelloWorld': True (Expected: True)
'HELLO': True (Expected: True)
'hello': True (Expected: True)
'Pelinal12.': False (Expected: False)
'Xyphen-9 send their regards.': False (Expected: False)
'...': False (Expected: False)
'Hello.': False (Expected: False)
'hello.': False (Expected: False)
'Hello world': False (Expected: False)
'': True (Expected: True)
'12345': True (Expected: True)
'Hello123': True (Expected: True)
'Yes!': False (Expected: False)
'No.': False (Expected: False)
'Maybe': False (Expected: False)
'None': True (Expected: True)
'123': True (Expected: True)
'TestStringWithoutSpace': True (Expected: True)
'This is a test string': False (Expected: False)
'Why?': False (Expected: False)
'OK': True (Expected: True)
'GO': True (Expected: True)
'DialogueGenericSharedInfo': True (Expected: True)
'Yes, clear my bounty. (<Global=GuildCost> gold)': False (Expected: False)
'